<h1 style="text-align: center;">Analyses d'un carrousel</h1>

# Préparation

## Imports des modules et configuration de base

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### # import du package `carrouselsAnalysis` et des classes, et tracks préconfigurées 

In [ ]:
# on importe le package complet `carrouselsAnalysis`
import carrouselsAnalysis

In [ ]:
#--------
# On importe la fonction de définition du style graphique
#--------
from carrouselsAnalysis import set_carrouselsAnalysis_graphic_style
#--------
# on modifie le style graphique de carrouselsAnalysis si nécessaire
#--------
#set_carrouselsAnalysis_graphic_style(titleSize=20,
#                                     xaxisSize=16,
#                                     yaxisSize=16)

In [ ]:
# on importe les classes Track et Carrousel du module `data`
from carrouselsAnalysis.carrousel import Track, Carrousel

TODO classe Carrousel:
- gestion d'un format de données par défaut pour le carrousel (FABISIS ou MG)

**NOTE pour l'utilisateur**:
- si nécessaire, changer le nom de la track préconfigurée à importer ci-dessous, après la mention `import`

In [ ]:
# on importe la configuration de track LilleL1 pré-configurée dans le module `tracks`
from carrouselsAnalysis.tracks import LilleL1

# Création de la track et d'une instance de carrousel

**NOTE pour l'utilisateur**:
- adapter ci-dessous le chemin des données ainsi que les éléments `source` et `contexte` du carrousel
- ajouter une référence (textuelle) de `build` s'il ne s'agit pas d'une version officielle

In [ ]:
# chemin des données à importer en local
chemin_data = './data/site/Lille_FLUENCE_DATA-2022-10-16&17__v1.xlsx'

# source des données. Typiquement 'FIVP' ou 'Site'
source = 'Site'

# contexte du carrousel. Typiquement 'mono-train' ou 'multi-trains'
contexte = 'multi-trains'

# référence du build utilisé (si non officiel, sinon la SoftwareVersion utilisée est celle extraite de l'OMAP)
build = '137b65'

**NOTE pour l'utilisateur**:
- changer la référence de la track préconfigurée utilisée ci-dessous si ce n'est pas LilleL1

In [ ]:
# création de la track LilleL1
track_1 = Track(LilleL1)

In [ ]:
# création du carrousel
c = Carrousel(track=track_1, source=source, context=contexte, dataPath=chemin_data, build=build)

## Import des données

utilisation des méthodes `get_movement`, `get_EB`

In [ ]:
# on importe les données de mouvement
c.get_movement()

TODO METHOD `get_movement`:
- gestion du format d'input (Fabisis ou MG) hérité du carrousel
- renommer la colonne stopStation en enlevant les `_0` et `_1` et en définissant une colonne `amont/aval`
- utiliser la colonne `SW version` pour définir un attribut de classe `version` qui pourra être hérité

In [ ]:
# on importe les données de freinage d'urgence
c.get_EB()

TODO METHOD `get_EB`:
- récupérer la `version` de la classe carrousel
- gestion du format d'input (FABISIS ou MG) hérité du carrousel

# Graphiques & indicateurs

TODO pour toutes les méthodes graphiques:
- gestion taille ticks

## Histogramme de précision globale de pilotage ATO

À FAIRE

## Arrêts ratés

### Synthèse

In [ ]:
c.synthese_arrets_rates(detailedState=True)

### Au global

In [ ]:
c.camembert_arrets_rates(resistivityState=['non-receptive',
                                           'partiellement'])

In [ ]:
c.histo_precision(xRange=(-1.3, 1.3))

In [ ]:
c.histo_precision_compare(xRange=(-2, 2),
                          dataCompare='CorrectDocking',
                         y='freq', globalFreq=True, style='bar')

In [ ]:
c.histo_precision_compare(x='absoluteDistanceSSP',
                          dataCompare='CorrectDocking',
                         y='freq', globalFreq=True, style='barstacked',
                         cumulative=True, xlim=(-0.1, 2),
                         bins=[0, 0.17, 0.3, 0.5, 0.6, 0.82, 2])

In [ ]:
c.histo_precision_filter(x='absoluteDistanceSSP',
                         dataFilter='CorrectDocking',
                         filterValue=False,
                         cumulative=True, y='freq', xlim=(-0.1, 2),
                        bins=[0, 0.17, 0.3, 0.5, 0.6, 0.82, 2])

In [ ]:
c.histo_precision_filter(x='DistanceSSP',
                         xRange=(-3, 2), y='freq', bins=40,
                        dataFilter='resistivityStateDual',
                        filterValue='non-receptive')

In [ ]:
c.histo_precision_filter(x='absoluteDistanceSSP',
                         xRange=(0,2), y='freq', cumulative=True,
                         xlim=(-0.1, 2),
                         bins=[0, 0.17, 0.3, 0.5, 0.6, 0.82, 2],
                         dataFilter='resistivityStateDetailed',
                         filterValue='non-receptive',
                        legend=True)

In [ ]:
sns.scatterplot(x='absoluteDistanceSSP', y='NVCommandSup40p',
               data=c.movements)

In [ ]:
dataviolin=c.movements[
    c.movements['DistanceSSP'] > -15]
list(dataviolin['TypeMovement'].unique())

In [ ]:
sns.catplot(data=dataviolin, x='DistanceSSP', y='StopTrackId',
            col='Direction', hue='resistivityStateDual',
            kind='violin', scale='count',
           inner='quartile', split=True, orient='h')
plt.show()

In [ ]:
sns.catplot(data=dataviolin, x='resistivityStateDual', hue='CorrectDocking',
            kind='count')
sns.countplot()
plt.xticks([0, 1], labels=['receptive', 'non-receptive'], rotation=None)
plt.show()

### Par station

In [ ]:
c.missed_stops_pct_by_station()

### Par train

In [ ]:
c.missed_stops_pct_by_train()

### Par période de temps

## Précision d'arrêt par station

### Précision moyenne

**Note pour l'utilisateur**:
- pour ajouter un intervalle de confiance basé sur l'écart type, utiliser `ci='sd'`

In [ ]:
c.trace_precision_station()

### Dispersion des arrêts par station

In [ ]:
c.trace_dispersion_station()

In [ ]:
fig = plt.figure(figsize=(8,10))
sns.catplot(data=c.movements, x='DistanceSSP', y='StopStation',
            hue='resistivityStateDual',
            kind='violin', scale='count',
           inner='quartile', split=True, orient='h', height = 20, aspect=0.4)
plt.xlim(1, -1)
plt.show()

## Précision d'arrêt par train

### Précision moyenne par train

In [ ]:
c.trace_precision_train()

### Dispersion par train

In [ ]:
c.trace_dispersion_train(figsize=(9.5,12))

## Temps de parcours

Note TODO:
- gestion du carrousel nominal seulement (par défaut) via les `PexMovements` de la track -> voir les mouvements nominaux à définir dans le carrousel
- et ordonnancement des movements nominaux dans l'ordre v1 puis v2

In [ ]:
c.trace_tps_parcours()

In [ ]:
c.trace_disp_tps_parcours(ylim=(40,110))

## Temps de parcours comparés mono-train vs multi-trains

À FAIRE

## Vitesse commerciale

À FAIRE: utilisation des `PexMovements` de la track

### Vitesse commerciale moyenne par voie

### Vitesse commerciale avec les temps de parcours mini

## Freinages d'urgence

### Répartition des freinages d'urgence par mouvement

#### y compris par type de mouvement (HLP, PEX, ??)

### Répartition des freinages d'urgence par KP

In [ ]:
c.trace_EB_by_KP(ylim=(0,15))

### Répartition des freinages d'urgence dans le temps

Note TODO:
- améliorer la représentation du format date dans les label d'axe
- permettre de choisir la période de temps représentée

In [ ]:
c.trace_EB_by_time(bins='30T', xticksRotation=90, ylim=(0,15))

### Répartition des freinages d'urgence par cause d'EB

À FAIRE

## Corrélations

### Corrélation entre nombre de FU par interstation et écart entre temps de parcours moyen et mini

## Brouillons de code

In [ ]:
running_times = pd.concat([running_times_FIVP, running_times_RC4])
running_times

In [ ]:
fig = plt.figure(figsize=(16,4))

sns.barplot(data=running_times, x='mouvement', y='Duree_MVT', hue='Source')
plt.title("Temps de parcours moyen")
plt.xticks(rotation=90)
plt.show()

In [ ]:
delta_running_times = running_times_RC4.merge(running_times_FIVP, on='mouvement', left_index=True)
delta_running_times['delta'] = delta_running_times['Duree_MVT_y'] - delta_running_times['Duree_MVT_x']
delta_running_times = delta_running_times[['mouvement', 'delta']]

In [ ]:
delta_running_times = delta_running_times[delta_running_times['delta'] > -50]

In [ ]:
fig = plt.figure(figsize=(16,4))
sns.barplot(data=delta_running_times, x='mouvement', y='delta', color='tab:blue',
           label=f"gain moyen par station: {delta_running_times['delta'].mean():.2f}s")
plt.title("Écart moyen de temps de parcours entre ATO Lot 3 RC4 sur site et FIVP 137B64",
         size=16)
plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,4))
sns.set_style()
sns.barplot(data=delta_running_times, x='mouvement', y='delta', hue='type_interstation', dodge=False)
plt.title("Écart moyen de temps de parcours entre ATO Lot 3 RC4 (mono-train) sur site et FIVP (40 trains) 137B64",
         size=16)
plt.xticks(rotation=90)
plt.ylabel('Delta temps de parcours')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,4))
delta_running_times_standard = delta_running_times[delta_running_times['type_interstation'] == 'Standard']
sns.barplot(data=delta_running_times_standard, x='mouvement', y='delta', color='tab:blue',
           label=f"gain moyen par station: {delta_running_times_standard['delta'].mean():.2f}s")
plt.title("Écart moyen de temps de parcours entre ATO Lot 3 RC4 (mono-train) sur site et FIVP 137B64 sur les interstation standard",
         size=16)
plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
tps_parcours = pd.read_excel('./tps_parcours_RC3_RC4.xlsx')
tps_parcours_137b64 = tps_parcours[['mouvement', '1.3.7.BETA64']].rename(columns={'1.3.7.BETA64': 'duree'})
tps_parcours_137b64['Version'] = '137b64_mono'

In [ ]:
fig = plt.figure(figsize=(16,16))
sns.barplot(data=tps_parcours_137b64, y='mouvement', x='duree', color='tab:blue', orient='h')

In [ ]:
running_times_FIVP = running_times_FIVP.rename(columns={'Duree_MVT': 'duree'})
running_times = pd.concat([running_times_FIVP, tps_parcours_137b64])
running_times[running_times['mouvement'] == "HEL1-MAR1"]

In [ ]:
fig = plt.figure(figsize=(16,4))

sns.barplot(data=running_times, x='mouvement', y='duree', hue='Version')
plt.title("Temps de parcours comparés mono vs multi trains en FIVP 137b64")
plt.xticks(rotation=90)
plt.legend(loc='upper left')
plt.show()

In [ ]:
delta_running_times = tps_parcours_137b64.merge(running_times_FIVP, on='mouvement', left_index=True)
delta_running_times['delta'] = delta_running_times['duree_y'] - delta_running_times['duree_x']
delta_running_times = delta_running_times[['mouvement', 'delta']]

In [ ]:
# delta_running_times = delta_running_times[delta_running_times['delta'] > -50]

In [ ]:
fig = plt.figure(figsize=(16,4))
sns.barplot(data=delta_running_times, x='mouvement', y='delta', color='tab:blue',
           label=f"gain moyen par station: {delta_running_times['delta'].mean():.2f}s")
plt.title("Écart moyen de temps de parcours entre mono train et multi trains FIVP 137b64",
         size=16)
plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,4))
sns.set_style()
sns.barplot(data=delta_running_times, x='mouvement', y='delta', hue='type_interstation', dodge=False)
plt.title("Écart moyen de temps de parcours entre mono train et multi trains FIVP 137b64",
         size=16)
plt.xticks(rotation=90)
plt.ylabel('Delta temps de parcours')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,4))
delta_running_times_standard = delta_running_times[delta_running_times['type_interstation'] == 'Standard']
sns.barplot(data=delta_running_times_standard, x='mouvement', y='delta', color='tab:blue',
           label=f"perte moyenne par station: {delta_running_times_standard['delta'].mean():.2f}s")
plt.title("Écart moyen de temps de parcours entre mono train et multi trains FIVP 137b64 sur les interstation standard",
         size=16)
plt.xticks(rotation=90)
plt.legend()
plt.show()